# Módulo 9: MLST (Tipificación de secuencias multilocus)

## Descripción general

La tipificación de secuencias multilocus (MLST) caracteriza los aislamientos de especies bacterianas utilizando las secuencias de fragmentos internos de (generalmente) siete genes house-keeping. Se utilizan fragmentos internos de aproximadamente 450-500 pb de cada gen, ya que estos se pueden secuenciar con precisión en ambas cadenas utilizando un secuenciador de ADN automatizado. Para cada gen house-keeping, las diferentes secuencias presentes dentro de una especie bacteriana se asignan como alelos distintos y, para cada aislamiento, los alelos en cada uno de los siete loci definen el perfil alélicos o tipo de secuencia (ST). Por lo tanto, cada aislamiento de una especie se caracteriza inequívocamente por una serie de siete números enteros que corresponden a los alelos en los siete loci de mantenimiento.

MLST se puede realizar utilizando SRST2 o MLST. Aprenderemos ambas herramientas en este módulo.

### Instalar condacolab

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

### Instalar programas

>**Nota**: En este módulo utilizaremos srst2 para la predicción de serotipos de *S. agalactiae*. Sin embargo, esta herramienta no funciona adecuadamente en Paython3. Por lo tanto, instalaremos Python2 y ejecutaremos la herramienta utilizando la versión python=2.7 con los comandos `!conda run -n py2_env`

In [ ]:
# Crear un ambiente en conda con la version 2.7 de python. El ambiente se llamará pyhton2 
!conda create -n py2_env python=2.7
#!apt-get install python2
!conda run -n py2_env python --version

In [ ]:
#Instalar srst2
!conda run -n py2_env conda install -c bioconda srst2 --yes

In [ ]:
# Compruebe si se srst2 se instaló correctamente
!conda run -n py2_env srst2 --help

In [ ]:
# Instalar mlst
!conda install -c conda-forge -c bioconda -c defaults mlst

In [ ]:
# Instalar any2fasta
!conda install -c bioconda any2fasta

In [ ]:
# Instalar blast
!conda install bioconda::blast

### Descargar datos

In [ ]:
!wget 

## Parte 1: Tipificación de secuencias multilocus usando SRST2

El programa [SRST2](https://github.com/katholt/srst2 ) (Short Read Sequence Typing for Bacterial Pathogens) está diseñado para tomar datos de secuencias de Illumina, una base de datos MLST y/o una base de datos de secuencias de genes (por ejemplo, genes de resistencia, genes de virulencia, etc.) e informar la presencia de ST y/o genes de referencia. Este programa realiza una detección rápida y precisa de genes y alelos a partir de lecturas de secuenciación corta WGS. SRST2 puede tipicar lecturas utilizando cualquier base de datos de secuencias y puede calcular tipos de secuencias combinatorias definidas en bases de datos tipo MLST.

*Lectura adicional*: https://genomemedicine.biomedcentral.com/articles/10.1186/s13073-014-0090-6

SRST2 requiere:

1. lecturas (usa lecturas pareadas: 17150_4#79_1.fastq.gz y 17150_4#79_2.fastq.gz)
2. Una base de datos de secuencias en un archivo fasta. Para MLST, esto significa un archivo fasta de todas las secuencias de alelos. Si desea asignar ST, también necesita un archivo tabulador que defina los perfiles de ST como una combinación de alelos. Puede recuperar estos archivos automáticamente desde pubmlst.org/data/ usando el script provisto a continuación:

### Paso 1: Descargue la base de datos de secuencias fasta y un archivo tab-delim de [PubMLST](https://pubmlst.org/organisms/streptococcus-agalactiae/) usando el comando:

In [ ]:
# Descargar la base de datos
!conda run -n py2_env getmlst.py --species 'Streptococcus pneumoniae'

Explore el contenido de su directorio de trabajo usando `ls`. Deberá tener los siguientes archivos:

- alelos_fasta,
- mlst_data_download_Streptococcus_pneumoniae_None.log,
- perfiles_csv
- Streptococcus_pneumoniae.fasta

Navegaremos a la carpeta que contiene los archivos pareados FASTQ ERR1795461_1.fastq.gz y ERR1795461_2.fastq.gz los cuales corresponden al run accession ERR1795461 del proyecto [PRJEB3084](https://www.ebi.ac.uk/ena/browser/view/PRJEB3084)	

Algunos datos importantes de la muestra:

- País de origen: Brasil
- Organismo: *Streptococcus pneumoniae*
- Instrumento: ILLUMINA
- Modelo del instrumento: Illumina MiSeq 
- Conteo de reads: 3627822
- Conteo de bases: 453477750 
- Nombre del centro: Wellcome Sanger Institute;SC 
- Diseño de librerías:  PAIRED
- Estrategia de librerías: WGS

### Paso 2: Para ejecutar SRST2 en una sola cepa usaremos el comando:

In [ ]:
# Ejecutar srst2
!conda run -n py2_env srst2 --input_pe ERR1795461_1.fastq.gz ERR1795461_2.fastq.gz --output ERR1795461 --log --mlst_db Streptococcus_pneumoniae.fasta --mlst_definitions profiles_csv --mlst_delimiter _

>Nota: Este proceso puede tomar mucho tiempo, por lo tanto, en la carpeta /content/Module_9/srst2/results/ podrá encontrar los resultados para analizar en los siguientes pasos.  

Una explicación de este comando es la siguiente:

`srst2` es la herramienta

`--input_pe` especifica que el archivo de entrada son lecturas finales emparejadas que son 17150_4#79_1.fastq.gz 17150_4#79_2.fastq.gz

`--output` especifica el archivo de salida 17150_4#79_test

`--mlst_db` especifica la base de datos Streptococcus_pneumoniae.fasta

`--mlst_definitions` especifica profiles_csv

`--mlst_delimiter` Caracteres que separan el nombre del gen del número de alelo en la base de datos MLST (predeterminado "-", como en arcc-1)

Dirijase a la carpeta `results`

In [ ]:
%cd /content/Module_9/srst2/results/

Ejecute el comando `ls -lh` para verificar el contenido de la carpeta.

In [ ]:
# Ver los resultados
!ls -lh

El archivo de salida de la ejecución de srst2 es `ERR1795461__mlst__Streptococcus_pneumoniae__results.txt`.

Entonces ejecute `cat` "ERR1795461__mlst__Streptococcus_pneumoniae__results.txt" para ver el contenido de este archivo.

La muestra ERR1795461 es MLST *****

### Para ejecutar SRST2 en múltiples cepas, ejecute el comando:

`--input_pe *.fastq.gz`: especifica que el archivo de entrada son varios archivos fastq.gz comprimidos.

In [ ]:
# No ejecutar
# Ejecutar srst2 para multiples los archivos
# srst2 --input_pe *.fastq.gz --output s.pneumo --log --mlst_db Streptococcus_pneumoniae.fasta --mlst_definitions profiles_csv --mlst_delimiter _

___

## Parte 2: Tipificación de secuencias multilocus usando mlst

[mlst](https://github.com/tseemann/mlst) escanea archivos contig contra los esquemas de escritura tradicionales de PubMLST

### Paso 1: para predecir el serotipo de una sola cepa (20280_5#33), usaremos el comando:

In [ ]:
# Correr mlst
!mlst --legacy --scheme spneumoniae 21127_1#30_output_contigs.fasta

Una explicación de este comando es la siguiente:

`mlst` es la herramienta

`--legacy` use la salida heredada anterior con la fila de encabezado de alelo (requiere --scheme)

`--scheme spneumoniae` especifica la especie (esquema pubmlst). Puede identificar el esquema usando el comando "docker_run staphb/mlst mlst --longlist"

`21127_1#30_output_contigs.fasta` archivo de entrada

Obtendrá esta salida impresa en la terminal

### Paso 2: Para ejecutar mlst en múltiples cepas, usaremos el comando:

In [ ]:
# Correr mlst para todos los archivos
mlst --legacy --scheme spneumoniae *.fasta > mlst.csv


Una explicación de este comando es la siguiente:

`mlst` es la herramienta

`--legacy` use la salida heredada anterior con la fila de encabezado de alelo (requiere --scheme)

`--scheme spneumoniae `especifica la especie (esquema pubmlst). Puede identificar el esquema usando el comando "docker_run staphb/mlst mlst --longlist"

`*.fasta` archivos de entrada

`>mlst.csv` especifica el archivo de salida

## BONO!

Si está trabajando con BASH en su computadora o en un HPC y tiene demasiados archivos, puede optimizar los comandos, los bucles (for loops) son muy útiles para grandes conjuntos de datos.

Aquí hay una manera de hacerlo.

Cree un nuevo script bash usando nano llamado `srst2.sh`

In [ ]:
#!/bin/bash
#Este script para hacer MLTS usando la herramienta srst2 desde fastq.gz recortados

for i in $(ls *_1.trimmed.fastq.gz); do
NAME=$(basename $i _1.trimmed.fastq.gz)
echo "$NAME"
j="${NAME}_1.trimmed.fastq.gz"
echo "$j"
k="${NAME}_2.trimmed.fastq.gz"
echo "$k"
srst2 --input_pe $j $k --output srst2output --log --mlst_db Streptococcus_pneumoniae.fasta --mlst_defin>
done

Cree un nuevo script bash usando nano llamado `mlst.sh`

In [ ]:
##para ejecutar con s.pneumonae
mlst --legacy --scheme spneumoniae *contigs.fasta > mlst_results.csv

##para ejecutar con s.agalactiae
mlst --legacy --scheme sagalactiae *contigs.fasta > mlst_results.csv